**问题**：
1. 按月进行统计的话，提货、分销、库存的数值很大，只要拟合不准，误差将是巨大的
2. 库存数据2018年只有每个月月底的库存量

In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from time import time
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 1000)

# 1 数据预处理

In [3]:
# 载入数据
order = pd.read_csv("../../data/level2/m111-sku-order.csv", sep=',', parse_dates=['order_date'])
dis = pd.read_csv("../../data/level2/m111-sku-dis.csv", sep=',', parse_dates=['dis_date'])
inv = pd.read_csv(
    "../../data/level2/m111-sku-inv.csv", sep=',', parse_dates=['period_wid']
).rename(columns={'period_wid': 'inv_date'})
category = pd.read_csv(
    "../../data/level2/item2category-from-3.csv", sep=','
).rename(columns={'sales_segment1_code': 'category'})

In [4]:
# 考虑的品类有：消毒柜、洗碗机、烟机、灶具、电热、净水机、燃热、饮水机
cates_considered = ['CRXDG', 'CRXWJ', 'CRYJ', 'CRZJ', 'DR', 'JSJ', 'RR', 'YSJ']

## 1.1 处理订单数据

In [5]:
# 取2017年至2018年区间中的数据（2019/03/01取数有164945条记录）
order = order.loc[(order.order_date >= '2017-01-01') & (order.order_date <= '2018-12-31')]

In [6]:
# 删除其他品类的数据（剩余记录数为161064）
order = order.join(category.set_index('item_code'), on='item_code', how='left')
order = order.loc[order.category.isin(cates_considered)]

In [7]:
# 划分数据集
df_test = order.loc[order.order_date >= '2018-11-01']  # 测试集
order = order.loc[order.order_date <= '2018-10-31']  # 训练和验证集

In [8]:
df_test['month'] = df_test.order_date.astype('str').apply(lambda x: x[:7])
df_test = df_test.groupby(['item_code', 'month'])[['qty']].sum()

In [9]:
# 每个产品每个月的提货量
order_month = order.copy()
order_month['month'] = order_month.order_date.astype('str').apply(lambda x: x[:7])
order_month = order_month.groupby(['item_code', 'month'])[['qty']].sum()

In [10]:
# 取对数
order_month['qty'] = np.log1p(order_month.qty)

In [11]:
order_month = order_month.unstack(level=-1).fillna(0)
order_month.columns = pd.date_range('2017-01-31', '2018-10-31', freq='M')

## 1.2 处理分销数据

In [12]:
# 取2017年至2018年区间中的数据（分销数据从2017年6月开始才可用，记录数为358247）
dis = dis.loc[(dis.dis_date >= '2017-06-01') & (dis.dis_date <= '2018-10-31')]

In [13]:
# 删除其他品类的数据（剩余记录数为343637）
dis = dis.join(category.set_index('item_code'), on='item_code', how='left')
dis = dis.loc[dis.category.isin(cates_considered)]

In [14]:
# 处理分销量为负数的情况，其中负数记录数为8766（备注：负数是有意义的，存在退货的情况）
# dis['qty'] = dis.qty.apply(lambda x: -x if x < 0 else x)
# dis = dis.loc[~(dis.qty < 0)]

In [15]:
# 每个产品每个月的分销量
dis_month = dis.copy()
dis_month['month'] = dis_month.dis_date.astype('str').apply(lambda x: x[:7])
dis_month = dis_month.groupby(['item_code', 'month'])[['qty']].sum()

In [16]:
# 删除分销为负数的记录
dis_month = dis_month.loc[~(dis_month.qty < 0)]

In [17]:
# 取对数
dis_month['qty'] = np.log1p(dis_month.qty)

In [18]:
dis_month = dis_month.unstack(level=-1).fillna(0)
dis_month.columns = pd.date_range('2017-06-30', '2018-10-31', freq='M')

In [19]:
dis_month = dis_month.reindex(order_month.index).fillna(0)

## 1.3 处理库存数据

In [20]:
# 取2017年至2018年区间中的数据（库存数据从2017年12月开始可用，记录数为138437）
inv = inv.loc[(inv.inv_date >= '2017-12-01') & (inv.inv_date <= '2018-10-31')]

In [21]:
# 删除其他品类的数据（剩余记录数为121249）
inv = inv.join(category.set_index('item_code'), on='item_code', how='left')
inv = inv.loc[inv.category.isin(cates_considered)]

In [22]:
# 取每月的最后一天作为当月的库存（剩余记录数为33698）
inv_lastday = inv.loc[inv.inv_date.isin(pd.date_range('2017-12-31', '2018-10-31', freq='M'))]

In [23]:
# 每个产品每月的库存量
inv_month = inv_lastday.copy()
inv_month['month'] = inv_month.inv_date.astype('str').apply(lambda x: x[:7])
inv_month = inv_month.groupby(['item_code', 'month'])[['qty']].sum()

In [24]:
# 删除库存为负数的记录
inv_month = inv_month.loc[~(inv_month.qty < 0)]

In [25]:
# 取对数
inv_month['qty'] = np.log1p(inv_month.qty)

In [26]:
inv_month = inv_month.unstack(level=-1).fillna(0)
inv_month.columns = pd.date_range('2017-12-31', '2018-10-31', freq='M')

In [27]:
inv_month = inv_month.reindex(order_month.index).fillna(0)

## 1.4 处理品类信息

In [28]:
category = category.set_index('item_code').reindex(order_month.index)

In [29]:
label_enc = LabelEncoder()
category['category'] = label_enc.fit_transform(category.category)

In [30]:
# onehot_enc = OneHotEncoder()
# category_onehot = onehot_enc.fit_transform(category).toarray()  # 不调用toarray方法前的结果是csr_matrix对象

## 1.5 得到每个品类每个月的提货数据

In [31]:
order_cate_month = order_month.reset_index()
order_cate_month['category'] = category.category.values
order_cate_month = order_cate_month.groupby('category')[order_month.columns].sum()

## 1.6 得到每个品类每个月的分销数据

In [32]:
dis_cate_month = dis_month.reset_index()
dis_cate_month['category'] = category['category'].values
dis_cate_month = dis_cate_month.groupby('category')[dis_month.columns].sum()

## 1.7 得到每个品类每个月的库存数据

In [33]:
inv_cate_month = inv_month.reset_index()
inv_cate_month['category'] = category['category'].values
inv_cate_month = inv_cate_month.groupby('category')[inv_month.columns].sum()

# 2 特征工程

In [34]:
def prepare_dataset(order, dis, inv, year, month, is_train=True, name_prefix=None):
    X = {}
    
    # 提货的统计特征（28个特征）
#     for i in [3, 6, 9, 12]:
#         dt = date(year, month, 1)
#         tmp = order[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月提货量
#         X['ord_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月提货量的平均一阶差分
#         X['ord_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月提货量的和（带衰减）
#         X['ord_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月提货量的平均值
#         X['ord_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月提货量的中位数
#         X['ord_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月提货量的最大值
#         X['ord_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月提货量的最小值
#         X['ord_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月提货量的标准差

    # 提货的统计特征
    for i in [3]:
        dt = date(year, month, 1)
        tmp = order[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月提货量
        X['ord_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月提货量的平均一阶差分
        X['ord_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月提货量的和（带衰减）
        X['ord_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月提货量的平均值
        X['ord_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月提货量的中位数
        X['ord_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月提货量的最大值
        X['ord_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月提货量的最小值
        X['ord_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月提货量的标准差
        
    # 分销的统计特征（21个特征）>_<|||
#     for i in [3, 6, 9]:
#         dt = date(year, month, 1)
#         tmp = dis[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月分销量
#         X['dis_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月分销量的平均一阶差分
#         X['dis_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月分销量的和（带衰减）
#         X['dis_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月分销量的均值
#         X['dis_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月分销量的中位数
#         X['dis_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月分销量的最大值
#         X['dis_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月分销量的最小值
#         X['dis_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月分销量的标准差

    # 分销的统计特征
    for i in [3]:
        dt = date(year, month, 1)
        tmp = dis[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月分销量
        X['dis_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月分销量的平均一阶差分
        X['dis_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月分销量的和（带衰减）
        X['dis_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月分销量的均值
        X['dis_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月分销量的中位数
        X['dis_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月分销量的最大值
        X['dis_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月分销量的最小值
        X['dis_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月分销量的标准差
        
    # 库存的统计特征（14个）>_<|||
#     for i in [3, 6]:
#         dt = date(year, month, 1)
#         tmp = inv[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月库存量
#         X['inv_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月库存量的平均一阶差分
#         X['inv_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月库存量的和（带衰减）
#         X['inv_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月库存量的均值
#         X['inv_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月库存量的中位数
#         X['inv_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月库存量的最大值
#         X['inv_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月库存量的最小值
#         X['inv_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月库存量的标准差
        
    # 提货天数特征（12个特征）
#     for i in [3, 6, 9, 12]:
#         dt = date(year, month, 1)
#         tmp = order[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_ord_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有提货的天数
#         X['last_ord_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有提货的天数
#         X['first_ord_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有提货的天数
        
    # 分销天数特征（9个特征）>_<|||
#     for i in [3, 6, 9]:
#         dt = date(year, month, 1)
#         tmp = dis[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_dis_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有分销的天数
#         X['last_dis_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有分销的天数
#         X['first_dis_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有分销的天数
        
    # 库存天数特征（6个特征）>_<|||
#     for i in [3, 6]:
#         dt = date(year, month, 1)
#         tmp = inv[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_inv_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有库存的天数
#         X['last_inv_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有库存的天数
#         X['first_inv_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有库存的天数
        
    # 前12个月的提货量
#     for i in range(1, 13):
#         if month - i <= 0:
#             start_dt = date(year - 1, month + 12 - i, 1)
#         else:
#             start_dt = date(year, month - i, 1)
#         X['ord_pre_%s' % i] = order[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()

    # 前3个月的提货量
    for i in range(1, 4):
        if month - i <= 0:
            start_dt = date(year - 1, month + 12 - i, 1)
        else:
            start_dt = date(year, month - i, 1)
        X['ord_pre_%s' % i] = order[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
        
    # 前9个月的分销量>_<|||
#     for i in range(1, 10):
#         if month - i <= 0:
#             start_dt = date(year - 1, month + 12 - i, 1)
#         else:
#             start_dt = date(year, month - i, 1)
#         X['dis_pre_%s' % i] = dis[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()

    # 前3个月分销量
    for i in range(1, 4):
        if month - i <= 0:
            start_dt = date(year - 1, month + 12 - i, 1)
        else:
            start_dt = date(year, month - i, 1)
        X['dis_pre_%s' % i] = dis[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
       
    # 前6个月的库存量>_<|||
#     for i in range(1, 7):
#         if month - i <= 0:
#             start_dt = date(year - 1, month + 12 - i, 1)
#         else:
#             start_dt = date(year, month - i, 1)
#         X['inv_pre_%s' % i] = inv[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
        
    X = pd.DataFrame(X)
    
    if is_train:
        start_dt = date(year, month, 1)
        y = order[pd.date_range(start_dt, periods=2, freq='M')].values
        return X, y
    
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
        
    return X

In [35]:
def get_pre_10_days(order, dis, inv, index, year, month):
    X = {} 
    start_dt, end_dt = date(year, month, 1), date(year, month, 10)
    
    # 每个产品M月前10天的提货量
    ord_tmp = order.loc[order.order_date.isin(pd.date_range(start_dt, end_dt, freq='D'))]
    ord_tmp = ord_tmp.groupby('item_code')[['qty']].sum()
    ord_tmp = ord_tmp.reindex(index).fillna(0)
    ord_tmp['qty'] = ord_tmp.qty.apply(lambda x: np.log1p(x) if x > 0 else 0)
    X['ord_pre_10_days'] = ord_tmp.values.ravel()
    
    # 每个产品M月前10天的分销量
    dis_tmp = dis.loc[dis.dis_date.isin(pd.date_range(start_dt, end_dt, freq='D'))]
    dis_tmp = dis_tmp.groupby('item_code')[['qty']].sum()
    dis_tmp = dis_tmp.reindex(index).fillna(0)
    dis_tmp['qty'] = dis_tmp.qty.apply(lambda x: np.log1p(x) if x > 0 else 0)
    X['dis_pre_10_days'] = dis_tmp.values.ravel()
    
    # 每个产品M月前10天的库存
#     inv_tmp = inv.loc[inv.inv_date.isin(pd.date_range(end=end_dt, periods=1, freq='D'))]
#     inv_tmp = inv_tmp.groupby('item_code')[['qty']].sum()
#     inv_tmp = inv_tmp.reindex(index).fillna(0)
#     inv_tmp['qty'] = inv_tmp.qty.apply(lambda x: np.log1p(x) if x > 0 else 0)
#     X['inv_pre_10_days'] = inv_tmp.values.ravel()
    
    X = pd.DataFrame(X)
    
    return X

## 2.1 准备训练集

In [36]:
train_month = [ 
#     '2017-05', 
#     '2017-06', 
#     '2017-07', 
#     '2017-08', 
    '2017-09', 
    '2017-10', 
    '2017-11', 
    '2017-12', 
    '2018-01', 
    '2018-03', 
    '2018-04', 
    '2018-05', 
    '2018-06', 
    '2018-07'
]

X_l, y_l = [], []
for month in train_month:
    y, m = int(month.split('-')[0]), int(month.split('-')[1])
    
    pre_10_days = get_pre_10_days(order, dis, inv, order_month.index, y, m)
    
    X_tmp, y_tmp = prepare_dataset(order_month, dis_month, inv_month, y, m)
    
#     X_tmp2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, y, m, is_train=False, name_prefix='cate')
#     X_tmp2.index = order_cate_month.index
#     X_tmp2 = X_tmp2.reindex(category.category).reset_index(drop=True)
    
#     X_tmp = pd.concat([X_tmp, X_tmp2, category.reset_index(drop=True)], axis=1)
    X_tmp = pd.concat([X_tmp, pre_10_days, category.reset_index(drop=True)], axis=1)
    X_tmp['pred_month'] = m
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
#     del X_tmp, y_tmp, X_tmp2
    del X_tmp, y_tmp
    gc.collect()
    
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

In [37]:
X_train.head()

,dis_diff_mean_pre_3,dis_max_pre_3,dis_mean_pre_3,dis_median_pre_3,dis_min_pre_3,dis_pre_1,dis_pre_2,dis_pre_3,dis_std_pre_3,dis_sum_decay_pre_3,ord_diff_mean_pre_3,ord_max_pre_3,ord_mean_pre_3,ord_median_pre_3,ord_min_pre_3,ord_pre_1,ord_pre_2,ord_pre_3,ord_std_pre_3,ord_sum_decay_pre_3,dis_pre_10_days,ord_pre_10_days,category,pred_month
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.454377,8.412055,5.106937,6.908755,0.000000,6.908755,8.412055,0.000000,4.486154,14.479604,0.0,0.000000,5,9
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.314682,6.629363,4.416923,6.621406,0.000000,0.000000,6.621406,6.629363,3.825170,11.329049,0.0,5.771441,2,9
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.524193,7.048386,4.617297,6.803505,0.000000,0.000000,6.803505,7.048386,4.000571,11.832348,0.0,0.000000,2,9
3,-3.697247,8.366835,5.253776,7.394493,0.000000,0.000000,8.366835,7.394493,4.575804,13.519691,-0.214463,9.016391,8.835211,8.901775,8.587465,8.587465,8.901775,9.016391,0.222076,23.902340,0.0,8.435766,2,9
4,0.608198,4.369448,3.248242,3.295837,2.079442,3.295837,4.369448,2.079442,1.145745,8.912688,-2.944439,5.888878,2.762258,2.397895,0.000000,0.000000,2.397895,5.888878,2.961299,6.928097,0.0,0.000000,2,9


In [38]:
X_train.shape

(15890, 24)

## 2.2 准备验证集

In [39]:
pre_10_days = get_pre_10_days(order, dis, inv, order_month.index, 2018, 9)

X_val, y_val = prepare_dataset(order_month, dis_month, inv_month, 2018, 9)

# X_val2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, 2018, 9, is_train=False, name_prefix='cate')
# X_val2.index = order_cate_month.index
# X_val2 = X_val2.reindex(category.category).reset_index(drop=True)

# X_val = pd.concat([X_val, X_val2, category.reset_index(drop=True)], axis=1)
X_val = pd.concat([X_val, pre_10_days, category.reset_index(drop=True)], axis=1)
X_val['pred_month'] = 9

# del X_val2
# gc.collect()

In [40]:
X_val.head()

,dis_diff_mean_pre_3,dis_max_pre_3,dis_mean_pre_3,dis_median_pre_3,dis_min_pre_3,dis_pre_1,dis_pre_2,dis_pre_3,dis_std_pre_3,dis_sum_decay_pre_3,ord_diff_mean_pre_3,ord_max_pre_3,ord_mean_pre_3,ord_median_pre_3,ord_min_pre_3,ord_pre_1,ord_pre_2,ord_pre_3,ord_std_pre_3,ord_sum_decay_pre_3,dis_pre_10_days,ord_pre_10_days,category,pred_month
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.853808,2.951269,0.000000,0.0,0.000000,8.853808,0.000000,5.111749,7.968427,0.0,0.0,5,9
1,0.000000,6.216606,2.072202,0.000000,0.000000,0.000000,6.216606,0.000000,3.589159,5.594945,-3.108303,6.216606,2.072202,0.000000,0.0,0.000000,0.000000,6.216606,3.589159,5.035451,0.0,0.0,2,9
2,0.000000,6.216606,2.072202,0.000000,0.000000,0.000000,6.216606,0.000000,3.589159,5.594945,-3.108303,6.216606,2.072202,0.000000,0.0,0.000000,0.000000,6.216606,3.589159,5.035451,0.0,0.0,2,9
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.800701,8.006701,5.202701,7.601402,0.0,7.601402,8.006701,0.000000,4.510226,14.807433,0.0,0.0,2,9
4,-0.618381,4.127134,3.071577,2.890372,2.197225,2.890372,2.197225,4.127134,0.977632,8.210853,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2,9


In [41]:
X_val.shape

(1589, 24)

## 2.3 准备测试集

In [42]:
pre_10_days = get_pre_10_days(order, dis, inv, order_month.index, 2018, 11)

X_test = prepare_dataset(order_month, dis_month, inv_month, 2018, 11, is_train=False)

# X_test2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, 2018, 11, is_train=False, name_prefix='cate')
# X_test2.index = order_cate_month.index
# X_test2 = X_test2.reindex(category['category']).reset_index(drop=True)

# X_test = pd.concat([X_test, X_test2, category.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test, pre_10_days, category.reset_index(drop=True)], axis=1)
X_test['pred_month'] = 11

# del X_test2
# gc.collect()

In [43]:
X_test.head()

,dis_diff_mean_pre_3,dis_max_pre_3,dis_mean_pre_3,dis_median_pre_3,dis_min_pre_3,dis_pre_1,dis_pre_2,dis_pre_3,dis_std_pre_3,dis_sum_decay_pre_3,ord_diff_mean_pre_3,ord_max_pre_3,ord_mean_pre_3,ord_median_pre_3,ord_min_pre_3,ord_pre_1,ord_pre_2,ord_pre_3,ord_std_pre_3,ord_sum_decay_pre_3,dis_pre_10_days,ord_pre_10_days,category,pred_month
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0,0,5,11
1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0,0,2,11
2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,6.398595,2.132865,0.000000,0.000000,0.000000,6.398595,0.000000,3.694231e+00,5.758735,0,0,2,11
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,7.601402,7.601402,7.601402,7.601402,7.601402,7.601402,7.601402,1.087792e-15,20.599800,0,0,2,11
4,-0.346574,2.890372,1.695865,2.197225,0.0,2.197225,0.0,2.890372,1.509001,4.538426,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0,0,2,11


In [44]:
X_test.shape

(1589, 24)

## 2.4 将 Categorical Feature 转化成 OneHot 向量

**备注**：
对于树模型而言，没有必要把 Categorical Feature 转化成 OneHot 向量，反而会使得准确率略微下降。

In [45]:
# train_len = len(X_train)
# val_len = len(X_val)

# tmp = pd.concat([X_train, X_val, X_test], axis=0)
# tmp = pd.get_dummies(tmp, columns=['category', 'pred_month'])

# X_train = tmp.iloc[:train_len]
# X_val = tmp.iloc[train_len:(train_len + val_len)]
# X_test = tmp.iloc[(train_len + val_len):]

# del tmp
# gc.collect()

# 3 训练和预测

In [46]:
print("[INFO] Start training and predicting...")
t0 = time()

params = {
    'num_leaves': 80, 
    'objective': 'regression', 
    'min_data_in_leaf': 200, 
    'learning_rate': 0.02, 
    'feature_fraction': 0.9, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 1, 
    'metric': 'l2', 
    'num_threads': 16
}

MAX_ROUNDS = 5000
pred_val = []
pred_test = []
cate_vars = []

for i in range(2):
    print('=' * 50)
    print("Step %d" % (i + 1))
    print('=' * 50)
    
    dtrain = lgb.Dataset(X_train, label=y_train[:, i], categorical_feature=cate_vars)
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain, categorical_feature=cate_vars)
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS, 
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    
    feat_imp = [("%s: %.2f" % x) for x in sorted(zip(X_train.columns, bst.feature_importance('gain')), key=lambda x: x[1], reverse=True)]
    print('\n'.join(feat_imp))
    pred_val.append(
        bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS)
    )
    pred_test.append(
        bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS)
    )
    
print("[INFO] Finished! ( ^ _ ^ ) V")
print("[INFO] Done in %f seconds." % (time() - t0))

[INFO] Start training and predicting...
Step 1


D:\Anaconda3\envs\py3_for_prac\lib\site-packages\lightgbm\basic.py:1040: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 3.03353	valid_1's l2: 3.46879
[100]	training's l2: 1.8342	valid_1's l2: 2.23764
[150]	training's l2: 1.63794	valid_1's l2: 2.08059
[200]	training's l2: 1.58706	valid_1's l2: 2.06795
[250]	training's l2: 1.5576	valid_1's l2: 2.07182
[300]	training's l2: 1.5314	valid_1's l2: 2.07406
Early stopping, best iteration is:
[197]	training's l2: 1.58912	valid_1's l2: 2.0668
ord_pre_10_days: 2549460.57
ord_pre_1: 271067.54
ord_sum_decay_pre_3: 170440.75
ord_mean_pre_3: 54727.35
ord_max_pre_3: 43502.19
pred_month: 12479.99
ord_min_pre_3: 7874.30
ord_median_pre_3: 7357.60
dis_pre_3: 5807.84
ord_pre_2: 4952.99
category: 3877.31
dis_std_pre_3: 3729.04
ord_pre_3: 3661.12
ord_std_pre_3: 3355.42
ord_diff_mean_pre_3: 3307.55
dis_diff_mean_pre_3: 2654.57
dis_max_pre_3: 2147.85
dis_pre_10_days: 2063.21
dis_pre_2: 1862.30
dis_min_pre_3: 1670.06
dis_mean_pre_3: 1460.60
dis_pre_1: 1032.96
dis_median_pre_3: 739.41
dis_sum_decay_

In [47]:
print("The MSE error of validation set is:", mean_squared_error(y_val, np.array(pred_val).transpose()))

The MSE error of validation set is: 3.0398676206330864


# 4 结果评估

## 4.1 技术指标

评价指标为**归一化加权均方根对数误差（Normalized Weighted Root Mean Squared Logarithmic Error, NWRMSLE）**，表达式如下：

$$
\text{NWRMSLE} = \sqrt{ \frac{\sum_{i=1}^{m}\omega_i \sum_{j=1}^{t}\left( \ln \left(\widehat{y}_j^{(i)} + 1 \right) -\ln \left(y_j^{(i)} + 1 \right)\right)^2}{t\cdot \sum_{i=1}^{m}\omega_i} }
$$

In [48]:
# 归一化加权均方根对数误差
err = (y_val - np.array(pred_val).transpose()) ** 2
err = err.sum(axis=1)
err = np.sqrt(err.sum() / 2 / len(y_val))
print("The NWRMSLE error of validation set is:", err)

The NWRMSLE error of validation set is: 1.7435216146159722


## 4.2 业务指标

业务指标的表达式如下：

$$
E(t, y) = \frac{\sum_{i=0}^{n} | y^{(i)} - t^{(i)} |}{\sum_{i=0}^{n} t^{(i)}}
$$

In [49]:
def error(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

In [50]:
def expm1_with_clip(x, l=0, r=35000):
    return np.clip(np.expm1(x), l, r)

### 4.2.1 验证集

In [51]:
df_val = pd.DataFrame(
    np.array(y_val), index=order_month.index, 
    columns=['2018-09', '2018-10']
).apply(np.expm1)

df_pred_val = pd.DataFrame(
    np.array(pred_val).transpose(), index=order_month.index, 
    columns=['2018-09', '2018-10']
).apply(expm1_with_clip).apply(np.floor)

In [52]:
# 验证集（9-10月）准确率（SKU）
m_error = error(df_val['2018-09'], df_pred_val['2018-09'])
m1_error = error(df_val['2018-10'], df_pred_val['2018-10'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 60.00%
The accuracy of 'M+1' order amount is: 33.86%


In [53]:
df_val = df_val.join(category, how='left')
df_pred_val = df_pred_val.join(category, how='left')

df_val = df_val.groupby('category')[['2018-09', '2018-10']].sum()
df_pred_val = df_pred_val.groupby('category')[['2018-09', '2018-10']].sum()

In [54]:
# 验证集（9-10月）准确率（品类）
m_error = error(df_val['2018-09'], df_pred_val['2018-09'])
m1_error = error(df_val['2018-10'], df_pred_val['2018-10'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 75.67%
The accuracy of 'M+1' order amount is: 41.73%


### 4.2.2 测试集

In [55]:
df_pred_test = pd.DataFrame(
    np.array(pred_test).transpose(), index=order_month.index, 
    columns=['2018-11', '2018-12']
).stack().to_frame('pred_qty')
df_pred_test.index.set_names(['item_code', 'month'], inplace=True)
df_pred_test['pred_qty'] = np.floor(expm1_with_clip(df_pred_test.pred_qty))

In [56]:
comp = df_test.join(df_pred_test, how='left').fillna(0).reset_index()
comp = comp.join(category, on='item_code', how='left')
comp.category.fillna('-1', inplace=True)

m_comp = comp.loc[comp['month'] == '2018-11']
m1_comp = comp.loc[comp['month'] == '2018-12']

m_comp_without_new = m_comp.loc[m_comp.category != '-1']
m1_comp_without_new = m1_comp.loc[m1_comp.category != '-1']

In [57]:
# 测试集（11-12月）准确率（带新品）（SKU）
# 备注：总共1653条提货记录，有120条提货记录是新品
m_error = error(m_comp['qty'], m_comp['pred_qty'])
m1_error = error(m1_comp['qty'], m1_comp['pred_qty'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 2.86%
The accuracy of 'M+1' order amount is: 5.22%


In [58]:
# 测试集（11-12月）准确率（不带新品）（SKU）
# 备注：总共1653条提货记录，有120条提货记录是新品
m_error = error(m_comp_without_new['qty'], m_comp['pred_qty'])
m1_error = error(m1_comp_without_new['qty'], m1_comp['pred_qty'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 2.94%
The accuracy of 'M+1' order amount is: 5.88%


In [59]:
comp_cate = comp.groupby(['category', 'month'])[['qty', 'pred_qty']].sum().reset_index()

m_comp = comp_cate.loc[comp_cate['month'] == '2018-11']
m1_comp = comp_cate.loc[comp_cate['month'] == '2018-12']

m_comp_without_new = m_comp.loc[m_comp.category != '-1']
m1_comp_without_new = m1_comp.loc[m1_comp.category != '-1']

In [60]:
# 测试集（11-12月）准确率（带新品）（品类）
m_error = error(m_comp['qty'], m_comp['pred_qty'])
m1_error = error(m1_comp['qty'], m1_comp['pred_qty'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 2.99%
The accuracy of 'M+1' order amount is: 5.99%


In [61]:
# 测试集（11-12月）准确率（不带新品）（品类）
m_error = error(m_comp_without_new['qty'], m_comp['pred_qty'])
m1_error = error(m1_comp_without_new['qty'], m1_comp['pred_qty'])
print("The accuracy of 'M' order amount is: %.2f%%" % ((1 - m_error) * 100))
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - m1_error) * 100))

The accuracy of 'M' order amount is: 3.07%
The accuracy of 'M+1' order amount is: 6.74%


In [62]:
m_comp['acc'] = 1 - (np.abs(m_comp.qty - m_comp.pred_qty) / m_comp.qty)
m1_comp['acc'] = 1 - (np.abs(m1_comp.qty - m1_comp.pred_qty) / m1_comp.qty)

D:\Anaconda3\envs\py3_for_prac\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\envs\py3_for_prac\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
m_comp.sort_values(by='acc', ascending=False)

,category,month,qty,pred_qty,acc
0,0,2018-11,17996.0,2036.0,0.113136
2,1,2018-11,13369.0,1143.0,0.085496
4,2,2018-11,146590.0,7033.0,0.047977
14,7,2018-11,144644.0,6606.0,0.045671
6,3,2018-11,128296.0,5089.0,0.039666
12,6,2018-11,267108.0,9107.0,0.034095
10,5,2018-11,373660.0,12679.0,0.033932
8,4,2018-11,979037.0,19925.0,0.020352
16,-1,2018-11,57027.0,0.0,0.000000


In [64]:
m1_comp.sort_values(by='acc', ascending=False)

,category,month,qty,pred_qty,acc
1,0,2018-12,37870.0,4696.0,0.124003
11,5,2018-12,479641.0,48116.0,0.100317
15,7,2018-12,166659.0,15363.0,0.092182
5,2,2018-12,313670.0,21727.0,0.069267
3,1,2018-12,24690.0,1502.0,0.060834
9,4,2018-12,1295903.0,78154.0,0.060309
7,3,2018-12,316960.0,18773.0,0.059228
13,6,2018-12,531017.0,24927.0,0.046942
17,-1,2018-12,396033.0,0.0,0.000000
